**Seleccionamos el grupo de préstamos más óptimo a titulizar según los límites establecidos**
- Por defecto lo que no venga marcado como límite es suceptible de cogerse para la titulización con un 100% (limit value por defecto 1.0)

In [5]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from pyspark.sql import functions as F, DataFrame
import datetime as dt
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import pyspark.sql.types as t
from decimal import Decimal
from pyspark.sql.functions import regexp_replace

In [6]:
from dataproc_sdk.dataproc_sdk_datiopysparksession.datiopysparksession import DatioPysparkSession
datioSparkSession = DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_datiopysparksession import datiopysparksession
dataproc = datiopysparksession.DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_schema.datioschema import DatioSchema
from dataproc_sdk.dataproc_sdk_datiofilesystem.datiofilesystem import DatioFileSystem

In [7]:
# para evitar problemas de tipología de datos
spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

# para evitar problemas de particiones
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")

In [8]:
# procesamiento de la cartera óptima en python
import pandas as pd
import numpy as np

# Configuracion

In [9]:
fecha = date.today() # por defecto la fecha de hoy (se actualizará en el proceso con la más reciente)

## Paths

### Paths in

In [10]:
root_path = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/'
root_path

'/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/'

In [11]:
# relación entre límites y a qué campo de Datio aplica
path_campos = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/limites/campos_datio/current/limites_camposDatio.csv'

### Paths out

In [12]:
path_out = root_path+'closing_date='+str(fecha)+'/cartera_titulizar'
path_out

'/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-22/cartera_titulizar'

In [13]:
path_out_csv = root_path+'closing_date='+str(fecha)+'/cartera_titulizar_csv'
path_out_csv

'/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-22/cartera_titulizar_csv'

## Columnas
Selección de columnas necesarias para acotar los limites

In [14]:
# clave unica de operación
key_facility = ['delta_file_id','delta_file_band_id','branch_id']
key_facility_micro = ['delta_file_id','delta_file_band_id'] # en MicroStrategy se quedan a este nivel

# clave del launchpad limites
key_limites = ['limit_escenario', 'limit_fecha', 'limit_portfolio_size']

# # columnas directas de la salida titulizaciones de joystick
# cols_lim = ['gf_ma_ead_amount','Total_Amount_EUR', # exposicion a default y consumo de capital
#             'clan_date','com_product', # fecha de los datos y tipo de producto(project finance o corporate loan)
#             'gf_pf_current_ratg_id','group_rating_sp', # análisis de rating individual para project finace y corporate loan
#             'project_country_desc','customer_country', # pais donde se lleva el proyecto y pais del cliente
#             'project_sector_desc','project_subsector_desc', # sector y subsector del proyecto
#             'g_asset_allocation_sector_desc','g_asset_allocation_subsec_desc', # sector y subsector de la actividad del cliente
#             'customer_country','group_country_desc',
#             'financial_product_desc','currency_id']
#             #'gf_pf_project_const_type',

# # columnas nuevas calculadas 
# cols_lim_c = ['non_ig_flag', # rating malos
#               'building_project_flag', # proyecto en construcción
#              ]

## Diccionarios
Se genera un diccionario para el grupo de facilities que entran en la cartera

### Tipos de Facility
tipología de las operaciones que entran en la titulizacion

In [15]:
d_tipo_facility = {'corporate_loan':'Corporate Facilities', # titulizacion tipo Corporate solo operaciones corporate
                 'project_finance':'Project Finance'} # titulizacion tipo Project solo operaciones project

### Limitación importe
sobre que campo aplica el % de los límites

In [16]:
d_imp_limit = {'individual':'importe_susceptible', # el % del limite se aplica sobre el importe susceptible de la facility
               'portfolio':'portfolio_size'} # el % del limite se aplica sobre el importe marcado en el portfolio size

# Funciones

### Porcentajes
para aquellos limites que su % va ligado a algún cálculo

In [17]:
def get_limit_value (df:DataFrame):
    df_l = df.withColumn('limit_value', 
                         F.when(F.col('limit_type')=='risk_retention',(F.lit(1) - F.col('limit_value')).cast('float')
                               ).otherwise(F.col('limit_value')))
    return df_l       

### Genéricas

In [18]:
# calculamos la fecha más reciente de la ruta tomando como campo de particion el pasado como parámetro
def last_partition (p_path:str, campo:str):
    
    datio_path = DatioFileSystem().get().qualify(p_path)
    fs = datio_path.fileSystem()
    path = datio_path.path()
    path_list = fs.listStatus(path)
    paths = [path.getPath().toString() for path in path_list] #listado de todos los paths de la ruta pasada
    
    l_fechas = [element.split(campo+'=')[1] for element in paths if campo in element] #listado de todas las fechas
    return max(l_fechas) # fecha mayor

In [19]:
# Formato fecha a formato string
def date_to_str(fecha: date, mascara: str = "%Y-%m-%d") -> str:
    try:
        return format(fecha.strftime(mascara))
    except ValueError as e:
        raise Exception(
            'Ha habido un error con la máscara ' + mascara + ' definida para la fecha ' + str(fecha) + ' \n {}'.format(
                str(e)))

In [20]:
# Formato string a formato fecha
def str_to_date(fecha: str, mascara: str = "%Y-%m-%d") -> datetime:
    try:
        return datetime.strptime(fecha, mascara)
    except ValueError as e:
        raise Exception(
            'Ha habido un error con la máscara ' + mascara + ' definida para la fecha ' + fecha + ' \n {}'.format(
                str(e)))

In [21]:
def get_fecha (fecha_ini:str, ndays:int, op:str='add'):
    if(op=='add'):
        d = fecha_ini + timedelta(days=ndays)
    else:
        d = fecha_ini - timedelta(days=ndays)
    return d

In [22]:
# get_fecha(2024-06-24,365) # datetime.date(2025, 6, 24)

In [23]:
# si es un valor numérico lo pasamos a int
import re

def value_cast(p):
    patron_numero = r'^(0|[1-9][0-9]*)$'
    pv=re.findall(patron_numero,p)
    if((pv==None) | (len(pv)==0)):
        return p
    else:
        return int(p)   

# 1. Pto Partida
- operaciones disponibles a carterizar
- limites establecidos para la titulizaciones

In [24]:
fecha_ejecucion = last_partition (root_path, 'closing_date')
print('Fecha de ejecución del modelo titulización:', fecha_ejecucion)
root_pathc = root_path + 'closing_date=' + str(fecha_ejecucion)
root_pathc

Fecha de ejecución del modelo titulización: 2024-10-22


'/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-22'

In [25]:
# actualizamos la fecha base
fecha = fecha_ejecucion

## Limites
los limites current

In [26]:
foto_limites= ['name_list_desc', 'limit_date']

In [27]:
# leemos los límites fijados y recalculamos el limit_value de aquellos necesarios
path_limites_only = root_pathc + '/limites'
limites_or = dataproc.read().parquet(path_limites_only)
limites = get_limit_value(limites_or)

In [28]:
#limites_or.show(1,False)

In [29]:
limites.show(1,False)

+-------------------------+----------+------------------+--------------+---------------------------------------+-------------+--------------+-----------+-------------------+--------------------+-----------+-----------+------------+-------------+--------+------------+
|name_list_desc           |limit_date|limit_type        |concept1_desc |concept1_value                         |concept2_desc|concept2_value|limit_value|corporate_loan_flag|project_finance_flag|limit_scope|active_flag|visual_order|complex_limit|id_limit|closing_date|
+-------------------------+----------+------------------+--------------+---------------------------------------+-------------+--------------+-----------+-------------------+--------------------+-----------+-----------+------------+-------------+--------+------------+
|escenario model verano IV|2024-06-24|customer_subsector|subsector_desc|Paper, plastic, metal & glass packaging|null         |null          |0.5        |1                  |0                   |po

In [30]:
# ya viene con el valor final 1-%launchpad
# limites.where(F.col('limit_type')=='risk_retention').show(1,False)

In [31]:
# limites.where(F.col('limit_type')=='sts_group').show(1,False)

In [32]:
escenario, date_titulizacion = [(x.name_list_desc,x.limit_date) for x in limites.select(*foto_limites).distinct().collect()][0]
print('escenario:',escenario)
print('fecha titulizacion',date_titulizacion)
print('numero de limites:',limites.count())

escenario: escenario model verano IV
fecha titulizacion 2024-06-24
numero de limites: 216


### Tipo titulizacion
marca las facilities que entran: solos las marcadas con Corporate/Project según corresponda

In [33]:
tipo_titulizacion = limites.where(F.col('limit_type')=='portfolio_type').select('corporate_loan_flag','project_finance_flag')
corporate_flag = tipo_titulizacion.select('corporate_loan_flag').collect()[0].corporate_loan_flag
project_flag = tipo_titulizacion.select('project_finance_flag').collect()[0].project_finance_flag

if(corporate_flag==1):
    tipo_f='corporate_loan'
    print('titulizacion de corporate loan')
if(project_flag==1):
    tipo_f='project_finance'
    print('titulizacion de project finance')   

titulizacion de corporate loan


In [34]:
# limite = 'risk_retention'
# limites.where(F.col('limit_type')==limite).show(50,False)

## Relación limite - campo
- cada límite el campo de la tabla de operaciones al que aplica

(*) es una tabla que se sube como un catálogo directamente al Sandbox

In [35]:
path_campos

'/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/limites/campos_datio/current/limites_camposDatio.csv'

In [36]:
limite_campo = spark.read.option('header','True').option('delimiter',';').csv(path_campos)
# limite_campo.show(5,False)

# depende de la tipología de la titulización cogemos una columna y otra para la referencia al campo Datio
if(corporate_flag==1):
    df_campos = limite_campo.drop('project_finance_column'
                                 ).withColumnRenamed('corporate_loan_column','campo_datio')
else:
    df_campos = limite_campo.drop('corporate_loan_column'
                                 ).withColumnRenamed('project_finance_column','campo_datio')
df_campos.show(5,False)

+--------------+----------------+------------+-----------+-----------+-----------+
|limit_type    |campo_datio     |complex_flag|header_flag|imp_limit  |null_values|
+--------------+----------------+------------+-----------+-----------+-----------+
|portfolio_size|gf_ma_ead_amount|0           |1          |limit_scope|0          |
|portfolio_date|clan_date       |0           |1          |limit_scope|0          |
|portfolio_type|com_product     |0           |1          |limit_scope|0          |
|risk_retention|Total_Amount_EUR|0           |0          |limit_scope|0          |
|non_ig        |non_ig_flag     |0           |0          |limit_scope|0          |
+--------------+----------------+------------+-----------+-----------+-----------+
only showing top 5 rows



In [37]:
# limite_campo.show(1,False)

In [38]:
# complex_flag si el limite afecta a 2 campos de Datio
# df_campos.where(F.col('complex_flag')==1).show(2,False)
# +-------------------------+-------------------------------------------------------------+------------+
# |limit_type               |campo_datio                                                  |complex_flag|
# +-------------------------+-------------------------------------------------------------+------------+
# |project_sector_subsector |project_sector_desc/project_subsector_desc                   |1           |

#### Listado de campos Datio
dejamos en una lista todos los campos que estan implicados en los límites

In [39]:
# dejamos el listado de todos los campos de Datio que se tienen en cuenta (Ej.project_sector_desc,project_subsector_desc)
cols_datio = [x.campo_datio.split('/')[0] for x in df_campos.select('campo_datio').distinct().collect()]
# cols_datio

### Fusionamos informacion Limites
Añadimos una columnas que por límite sepamos el campo Datio al que hace referencia

In [40]:
limites_total = limites.join(df_campos,['limit_type'],'left').dropDuplicates(
).withColumn('imp_limit', F.when(F.col('imp_limit')=='limit_scope', F.col('limit_scope')).otherwise(F.col('imp_limit')))

limites_total.show(5,False)
# si es un limite compuesto en la columna campo Datio campo1/campo2

+------------------+-------------------------+----------+--------------+----------------------------+-------------+--------------+-----------+-------------------+--------------------+-----------+-----------+------------+-------------+--------+------------+------------------------------+------------+-----------+----------+-----------+
|limit_type        |name_list_desc           |limit_date|concept1_desc |concept1_value              |concept2_desc|concept2_value|limit_value|corporate_loan_flag|project_finance_flag|limit_scope|active_flag|visual_order|complex_limit|id_limit|closing_date|campo_datio                   |complex_flag|header_flag|imp_limit |null_values|
+------------------+-------------------------+----------+--------------+----------------------------+-------------+--------------+-----------+-------------------+--------------------+-----------+-----------+------------+-------------+--------+------------+------------------------------+------------+-----------+----------+-----

In [41]:
# limite = 'risk_retention'
# limites_total.where(F.col('limit_type')==limite).show(50,False)

## Operaciones
la ultima foto disponible

In [42]:
path_facilities = root_pathc + '/facilities'
facilities_0 = dataproc.read().parquet(path_facilities)

In [43]:
path_facilities

'/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/cartera_optima/closing_date=2024-10-22/facilities'

In [44]:
#facilities.groupBy('com_product').count().show(truncate=False)

In [45]:
#d_tipo_facility[tipo_f]

### Facilities tipo
solo cogemos las facilities del tipo de titulizacion

In [46]:
facilities = facilities_0.where(F.col('com_product')==d_tipo_facility[tipo_f])

In [47]:
if(facilities.count()==facilities.select(*key_facility).distinct().count()):
    print('Datos a nivel facility')
fecha_facilities = date_to_str([x.data_date for x in facilities.select('data_date').distinct().collect()][0])
    
print('numero de operaciones de base:', facilities.select(*key_facility).distinct().count())
print('fecha foto facilities:',fecha_facilities)

Datos a nivel facility
numero de operaciones de base: 1023
fecha foto facilities: 2024-10-21


### Añadimos Columnas Traza
- excluded: operación excluida, se marca con flag 1/0 para indicar si se excluye por los límites
- exclusion_limit: limite que marca su exclusión de la cartera óptima

Se inicializan a excluded=0 y exclusion_limit=''

In [48]:
cols_traza = ['excluded','exclusion_limit']

In [49]:
facilities_t = facilities.withColumn('excluded', F.lit(0)
                        ).withColumn('exclusion_limit', F.lit(''))

In [50]:
# sorted(facilities.columns)

In [51]:
facilities_t.show(2,False)

+-----------------------+-------------+------------------+---------+------------------------+----------------------+--------------------------+--------------+------------+-----------+-----------------+---------------+----------------------------+-----------+------------------------+------------------+------------------------+---------------------------+------------------------------+-------------------------------+---------------+-------------------------+------------------+---------------+--------------------+------------------------------+------------------------------+----------------+--------------------+-----------------+---------------------+---------------------+-------------------------+---------------+-----------------------------+--------------------------+------------------------+----------------------------+---------------------+--------------------+----------------------+---------------------------+--------------------------+--------------------------+----------------+----

## Portfolio Size
el importe total que se quiere titulizar

In [52]:
portfolio_size = [x.limit_value for x in limites_total.where(F.col('limit_type')=='portfolio_size').select('limit_value').collect()][0]
print('Importe a portfolio size total:', portfolio_size)

Importe a portfolio size total: 2000000000.0


# 2. Importe Susceptible (importe base)
Por cada facility: Calculamos aplicando fórmula de la hoja táctica, que parte de la facility se podría titulizar

## Aplicamos fórmula

**importe_susceptible = (min(ead_regulatorio, (saldo vivo+importe disponible*CCF))) - (importe_titulizado/risk_retention)**
- ead_regulatorio -> gf_ma_ead_amount
- saldo vivo -> bbva_drawn_eur_amount (saldo dispuesto - amortizado). Es directamente el dato del dispuesto en euros
- importe disponible -> bbva_available_eur_amount (importe en oficina en euros)
- CCF : dentro del launchpad de limites (limit_type=='ccf'). Es el factor de conversión con el que se calcula el EAD
- importe titulizado -> gf_facility_securitization_amount
- risk retention : dentro del launchpad de limite (limit_type=='risk_retention'). Ya está calculado su valor con (1 - %del excel), que sería el % a aplicar

= +MIN(AU81;U81+75%*V81)-SI.ERROR(BUSCARV( 𝑍81;′𝑉𝑒𝑟𝑎𝑛𝑜𝐼𝑅𝑅′! B:$C;2;0)/80%;0) = min(EADReg, Saldo Vivo BBVA + Importe Disponible BBVA_euros * 0.75) - (Reference Obligation Notional Amount/0.8)

### Límites necesarios 
Incluimos columnas limites necesarios

In [53]:
# como hay limites que se aplican directamente en alguna fórmula los identificamos para saber que ya se han aplicado
lim_usados = []

In [54]:
lim_imp = ['risk_retention', 'ccf']
lim_usados = lim_usados + lim_imp

In [55]:
l = [(x.limit_type, x.limit_value) for x in limites_total.select('limit_type','limit_value').where(F.col('limit_type').isin(*lim_imp)).collect()]
l

[('risk_retention', 0.800000011920929), ('ccf', 0.75)]

In [56]:
facilities_t1 = facilities_t.withColumn('limit_escenario',F.lit(escenario)
                           ).withColumn('limit_fecha',F.lit(date_titulizacion))
for e in l:
    facilities_t1 = facilities_t1.withColumn('limit_'+e[0],F.lit(e[1]))

In [57]:
facilities_t1.select('limit_ccf','limit_risk_retention').show(2,False)

+---------+--------------------+
|limit_ccf|limit_risk_retention|
+---------+--------------------+
|0.75     |0.800000011920929   |
|0.75     |0.800000011920929   |
+---------+--------------------+
only showing top 2 rows



### Cálculo del importe
Se incluye una nueva columna con el importe

importe_susceptible = (min(ead_regulatorio, (saldo vivo+importe disponible*CCF))) - (importe_titulizado/risk_retention).

Lo dividimos en partes:
- importe1 = saldo vivo+importe disponible*CCF
- importe2 = importe_titulizado/risk_retention
- importe_susceptible = min(ead_regulatorio,importe1) - importe2

In [58]:
facilities_f0 = facilities_t1.withColumn('importe1', F.col('bbva_drawn_eur_amount')+(F.col('bbva_available_eur_amount')*F.col('limit_ccf'))
                                           ).withColumn('importe2', F.col('gf_facility_securitization_amount')/F.col('limit_risk_retention')
                                           ).withColumn('importe_susceptible', F.least(F.col('gf_ma_ead_amount'),F.col('importe1')) - F.col('importe2'))

In [59]:
id_test = '366005' # '815974' #'415725'

cols_f0 = ['bbva_drawn_eur_amount','bbva_available_eur_amount','limit_ccf','importe1',
          'gf_facility_securitization_amount','limit_risk_retention','importe2',
          'gf_ma_ead_amount','importe_susceptible']

facilities_f0.select(*key_facility,*cols_f0).where(F.col('delta_file_id')==id_test).show(5,False)

+-------------+------------------+---------+---------------------+-------------------------+---------+--------+---------------------------------+--------------------+--------+----------------+-------------------+
|delta_file_id|delta_file_band_id|branch_id|bbva_drawn_eur_amount|bbva_available_eur_amount|limit_ccf|importe1|gf_facility_securitization_amount|limit_risk_retention|importe2|gf_ma_ead_amount|importe_susceptible|
+-------------+------------------+---------+---------------------+-------------------------+---------+--------+---------------------------------+--------------------+--------+----------------+-------------------+
+-------------+------------------+---------+---------------------+-------------------------+---------+--------+---------------------------------+--------------------+--------+----------------+-------------------+



In [60]:
print('Facilities totales:', facilities_f0.count())
print('Facilities con importe a titulizar:', facilities_f0.where(F.col('importe_susceptible')>0).count())

Facilities totales: 1023
Facilities con importe a titulizar: 917


In [61]:
# facilities_f1.where(F.col('delta_file_id')=='415725').show(5,False)
# tramo 1: 24.750.854,13
# tramo 3: 366.875,99

**(*)otra formula: min((saldo_vivo - Titulizado/risk_retention), (ead/risk_retention))**
- importe1a = saldo_vivo - Titulizado/risk_retention . (*) no sé si ccf o risk_retention, en la hoja aplica directamente un 0.8
- importe2a = ead/risk_retention
- importe_susceptible = min(importe1a,importe2a)

In [62]:
# NUEVOS CAMPOS: para esta fórmula si fueran necesarios
# facilities_f1 = facilities_f0.withColumn('importe1_v1', (F.col('bbva_drawn_eur_amount') - (F.col('gf_facility_securitization_amount')/F.col('limit_risk_retention')))
#                                     ).withColumn('importe2_v1', F.col('gf_ma_ead_amount')/F.col('limit_risk_retention')
#                                     ).withColumn('importe_susceptible_v1',F.least('importe1_v1','importe2_v1'))

In [63]:
# cols_fa = ['bbva_drawn_eur_amount','gf_facility_securitization_amount','limit_risk_retention','importe1_v1',
#           'gf_ma_ead_amount','importe2_v1','importe_susceptible_v1']

# facilities_f1.select(*key_facility,*cols_fa).where(F.col('delta_file_id')==id_test).show(5,False)                                            

In [64]:
# TEST
# cls = list(set(key_facility + cols_f0 + cols_fa))
# facilities_f1.where(F.col('importe_susceptible_v1')!=F.col('importe_susceptible')).count() # 886
# facilities_f1.where(F.col('importe_susceptible_v1')!=F.col('importe_susceptible')).select(*cls).orderBy(*key_facility).show(5,False)
# facilities_f1.where((F.col('importe_susceptible_v1')>0) & (F.col('importe_susceptible')==0)).count() # 0
# facilities_f1.where((F.col('importe_susceptible')>0) & (F.col('importe_susceptible_v1')==0)).count() # 379

# 3. Límites Individuales
Aplicamos limites ámbito individual: min(%limit_value) 

- añadimos columna con el % que se aplica por cada límite individual
- añadimos columna con el límite individual más restrictivo (el de menor % en el límite)
- añadimos columna con el % del valor del límite individual más restrictivo (menor %)
- añadimos columna con el importe titulizable de cada facility : importe susceptible a titulizar * min(%individual)

In [65]:
limites_ind = limites_total.where(F.col('limit_scope')=='individual')
limites_ind.select('limit_type','concept1_desc','campo_datio','limit_scope').distinct().orderBy('limit_type').show(limites_ind.count(),False)

if(limites_ind.where(F.col('complex_limit')==1).count()>0):
    print('existen filtros complejos: definición de 2 niveles de conceptos')
    limites_ind.where(F.col('complex_limit')==1
                    ).select('limit_type','concept1_desc','concept2_desc','campo_datio','limit_scope').distinct().orderBy('limit_type').show(20,False)

+-------------------+-----------------+------------------------------+-----------+
|limit_type         |concept1_desc    |campo_datio                   |limit_scope|
+-------------------+-----------------+------------------------------+-----------+
|bei                |bei_flag         |delta_file_id                 |individual |
|excluded_facilities|facility_id      |delta_file_id                 |individual |
|maturity_min       |num_days         |expiration_date               |individual |
|rating_rga_esl     |rating_large     |gf_ma_expanded_master_scale_id|individual |
|rating_sp          |rating_group     |group_rating_sp               |individual |
|risk_retention     |risk_bbva_percent|Total_Amount_EUR              |individual |
|sts_payment        |sts_flag         |sts_payment_flag              |individual |
|sts_rw_modelo      |sts_flag         |sts_sm_rw_flag                |individual |
+-------------------+-----------------+------------------------------+-----------+

exi

### Diccionario
correspondencia entre: limite - campo Datio

In [66]:
# limit_ind.show(2,False)

In [67]:
# limites simples
lista =limites_ind.select('limit_type','campo_datio').collect()
dict_lim_ind = {row['limit_type']: row['campo_datio'] for row in lista}

In [68]:
dict_lim_ind

{'rating_rga_esl': 'gf_ma_expanded_master_scale_id',
 'rating_sp': 'group_rating_sp',
 'excluded_facilities': 'delta_file_id',
 'risk_retention': 'Total_Amount_EUR',
 'maturity_min': 'expiration_date',
 'sts_rw_modelo': 'sts_sm_rw_flag',
 'bei': 'delta_file_id',
 'sts_payment': 'sts_payment_flag'}

In [69]:
# dict_lim_ind

In [70]:
# dict_lim_ind_p

### Columnas valor nulo
para campos a nulos que % del limite se aplica. Por defecto limite aplica a 0% si la columna tiene valor nulo
-  viene marcado en el fichero limite-campo

In [71]:
l =limites_ind.select('limit_type','null_values').collect()
dict_lim_ind_nul = {row['limit_type']: row['null_values'] for row in l}
# dict_lim_ind_nul

### Limites por Facility
- generamos columna con lista de limites aplicados
- generamos las columnas con el valor de los limites

In [72]:
facilities_add = facilities_f0.withColumn('limits_applied',F.lit('')) # tomamos de base las facilities y vamos añadiendo columnas y comprobaciones
varios_campos = 0 # partimos de la base que sólo afecta a un campo de Datio
limits_indv = []
cols_facility = facilities_add.columns

for k,v in dict_lim_ind.items():
    print('*** limite analizado:',k)
    print('campo de la facility al que aplica:',v)
    
    if('/' in v): # limite con varios campos viene con este separador en el catálogo de campos
        varios_campos = 1
        
    # CASO BASE: casi todos lo límites aplican a un solo campo
    if (varios_campos == 0):
        df_limite1 = limites_ind.where(F.col('limit_type')==k
                                            ).withColumn('concepto_valor',F.when(F.col('complex_limit')==1,F.col('concept2_value')).otherwise(F.col('concept1_value'))
                                            ).select('concepto_valor','limit_value'
                                            ).withColumnRenamed('concepto_valor' ,v
                                            ).withColumnRenamed('limit_value' ,'limit_'+k
                                            ).withColumn('limit_'+k,F.col('limit_'+k).cast("float")
                                            ).withColumn('limit_apply', F.lit('limit_'+k))
        
        
        # limite generico para todas las facilities
        if((df_limite1.count()==1)&(df_limite1.select(v).collect()[0][v]=='total')):
            facilities_add = facilities_add.withColumn('limit_'+k,F.lit(df_limite1.select('limit_'+k).collect()[0]['limit_'+k]).cast("float")
                                                      ).withColumn('limit_apply', F.lit('limit_'+k))
            print('limit simple con limit value total:','limit_'+k)
        
        # limite según categoria - valor
        else:
            facilities_add = facilities_add.join(df_limite1, [v],'left').fillna(1.0).fillna({'limit_apply':''}) # si algún valor no vienen marcado en el límite se aplica un valor del 100% (1,0)
            # valor del limite para columnas nulas
            n0 = facilities_add.where(F.col(v).isNull()).count()
            if(n0>0): # si hay columnas nulas para ese campo
                print('facilities con columna:',v,'nula:',facilities_add.where(F.col(v).isNull()).count())
                print('% de limite',k,'para estas facilities:',dict_lim_ind_nul[k])
                facilities_add = facilities_add.withColumn('limit_'+k,F.when(F.col(v).isNull(),dict_lim_ind_nul[k]).otherwise(F.col('limit_'+k))) # pongo el limite establecido para las columnas nulas 
    
    #CASO COMPLEJO: aplica a varios campos 
    else:
        v1,v2 = v.split('/')
        
        df_limite2 = limites_ind.where(F.col('limit_type')==k
                                            ).select('concept1_value','concept2_value','limit_value'
                                            ).withColumnRenamed('concept1_value' ,v1
                                            ).withColumnRenamed('concept2_value' ,v2
                                            ).withColumnRenamed('limit_value' ,'limit_'+k
                                            ).withColumn('limit_'+k,F.col('limit_'+k).cast("float")
                                            ).withColumn('limit_apply', F.lit('limit_'+k))
        
        # limite generico para todas las facilities
        if((df_limite2.count()==1)&(df_limite2.select(v1).collect()[0][v1]=='total')):

            if((df_limite2.count()==1)&(df_limite2.select(v2).collect()[0][v2]=='total')):
                facilities_add = facilities_add.withColumn('limit_'+k,F.lit(df_limite2.select('limit_'+k).collect()[0]['limit_'+k]).cast("float")
                                                          ).withColumn('limit_apply', F.lit('limit_'+k))
            else:
                facilities_add = facilities_add.join(df_limite2, [v2],'left').fillna(1.0).fillna({'limit_apply':''}) # si algún valor no vienen marcado en el límite se aplica un valor del 100% (1,0)

        # limite según categoria - valor
        else:
            if((df_limite2.count()==1)&(df_limite2.select(v2).collect()[0][v2]=='total')):
                facilities_add = facilities_add.join(df_limite2, [v1],'left').fillna(1.0).fillna({'limit_apply':''}) # si algún valor no vienen marcado en el límite se aplica un valor del 100% (1,0)
            else:
                facilities_add = facilities_add.join(df_limite2, [v1,v2],'left').fillna(1.0).fillna({'limit_apply':''}) # si algún valor no vienen marcado en el límite se aplica un valor del 100% (1,0)   
                # valor del limite para columnas nulas
                n1 = facilities_add.where(F.col(v2).isNull()).count()
                if(n1>0): #si hay valores nulos para la columna v2
                    print('facilities con columna:',v2,'nula:',facilities_add.where(F.col(v2).isNull()).count())
                    print('% de limite',k,'para estas facilities:',dict_lim_ind_nul[k])
                    facilities_add = facilities_add.withColumn('limit_'+k,F.when(F.col(v2).isNull(),dict_lim_ind_nul[k]).otherwise(F.col('limit_'+k))) # pongo el limite establecido para las columnas nulas 
    
    # añadimos al campo de limites aplicados el procesado que vendrá con valor cuando los limites han coincidido
    facilities_add = facilities_add.withColumn('limits_applied',F.when(F.col('limit_apply')!='',F.concat(F.col('limits_applied'),F.lit(','),F.col('limit_apply'))
                                                                      ).otherwise(F.col('limits_applied'))).drop('limit_apply'
                                   ).withColumn('exclusion_limit', F.when(F.col('limit_'+k)==0,F.concat(F.lit(k),F.lit(', '),F.col('exclusion_limit'))
                                                                    ).otherwise(F.col('exclusion_limit')))
    limits_indv.append('limit_'+k)
    varios_campos = 0

*** limite analizado: rating_rga_esl
campo de la facility al que aplica: gf_ma_expanded_master_scale_id
*** limite analizado: rating_sp
campo de la facility al que aplica: group_rating_sp
*** limite analizado: excluded_facilities
campo de la facility al que aplica: delta_file_id
*** limite analizado: risk_retention
campo de la facility al que aplica: Total_Amount_EUR
limit simple con limit value total: limit_risk_retention
*** limite analizado: maturity_min
campo de la facility al que aplica: expiration_date
*** limite analizado: sts_rw_modelo
campo de la facility al que aplica: sts_sm_rw_flag
*** limite analizado: bei
campo de la facility al que aplica: delta_file_id
*** limite analizado: sts_payment
campo de la facility al que aplica: sts_payment_flag


In [73]:
# tipos de agrupaciones de limites
# facilities_add.groupBy('limits_applied').count().show(truncate=False)

In [74]:
campos = list(dict_lim_ind.values())
facilities_add.select(*key_facility,'limits_applied',*limits_indv,*campos).show(5,False)
#.where(F.col('limit_rating_sp')<1).show(5,False)

+-------------+------------------+---------+------------------------------------------+--------------------+---------------+-------------------------+--------------------+------------------+-------------------+---------+-----------------+------------------------------+---------------+-------------+----------------+---------------+--------------+-------------+----------------+
|delta_file_id|delta_file_band_id|branch_id|limits_applied                            |limit_rating_rga_esl|limit_rating_sp|limit_excluded_facilities|limit_risk_retention|limit_maturity_min|limit_sts_rw_modelo|limit_bei|limit_sts_payment|gf_ma_expanded_master_scale_id|group_rating_sp|delta_file_id|Total_Amount_EUR|expiration_date|sts_sm_rw_flag|delta_file_id|sts_payment_flag|
+-------------+------------------+---------+------------------------------------------+--------------------+---------------+-------------------------+--------------------+------------------+-------------------+---------+-----------------+----

In [75]:
# TEST de los limites aplicados
# n_filas = 5
# for k,v in dict_lim_ind.items():
#     l = 'limit_'+k
#     print('limite testeado:',l)
#     facilities_add.groupBy(l,v).count().orderBy(l,v).show(n_filas,False)

In [76]:
# Rating completo
# facilities_add.groupBy('limit_rating_sp','g_lmscl_internal_ratg_type').count().orderBy('limit_rating_sp','g_lmscl_internal_ratg_type').show(50,False)

# tipos de rating:
# - g_lmscl_internal_ratg_type es el rating interno formato largo
# - gf_pf_current_ratg_id es el de acreditadas (para project finance): viene marcado que sea este campo para titulizaciones project finance, es el de s&p
# - group_rating_sp es el del grupo

In [77]:
# sorted(facilities_add.columns)

### Importe titulizable
por cada facility, calculamos:
- limite_individual: menor de sus limites, para quedarnos con el % más restrictivo
- importe titulizable = importe susceptible * limite_individual
    -(*)para limites cuyo %vaya sobre el portofolio (rating_esg) se limita el importe susceptible al importe máximo individual
- candidata: facilities con importe titulizable
marcamos si es excluida por limites:
- 'excluded'=1, si algun limite 0 y son facilities excluidas por limites
- 'exclusion_limit': el/los limites que son excluyentes porque su porcentaje marcado en el launchpad es 0

#### Importe máximo limite
si es scope individual pero imp_limit portfolio, se trata a parte
- viene marcado en el fichero limite-campo
- imp_limit: sobre qué importe se calcula el % del limite (si es portfolio en un limite individual,
    el % se calcula sobre el portofolio_size para ver si cumple el limite o no el importe titulizable de la facility)

In [78]:
# identifico si hay limites individuales con imp_limit portfolio para tratarlos a parte (ya que su máximo va sobre portfolio_size)
l_pr = [l.limit_type for l in limites_ind.where(F.col('imp_limit')!='individual').select('limit_type').distinct().collect()]
dict_lim_ind_p ={}
facilities_add_p = facilities_add.withColumn('imp_maximo_individual', F.lit(portfolio_size)
                                            ).withColumn('limit_individual_p',F.lit(1.0))

# si existe limite con importe_maximo_individual
if(len(l_pr)>0):
    for k in l_pr:
        print('limite con importe sujeto a portfolio size:', k)
        dict_lim_ind_p[k]=dict_lim_ind[k]
        if('limit_'+k in limits_indv):
            limits_indv.remove('limit_'+k) # quitamos del cálculo del minimo limite individual
        facilities_add_p = facilities_add_p.withColumn('imp_maximo_'+k, (F.col('limit_'+k)*portfolio_size).cast("float"))#calculo ese importe máximo
        
    if(len(l_pr)==1):
        print('solo un limite individual con limite de importe portfolio')
        facilities_add_p = facilities_add_p.withColumn('imp_maximo_individual', F.col('imp_maximo_'+k)
                                                      ).withColumn('limit_individual_p',F.col('limit_'+k)) # el importe y limite más restrictivo de los limites % sobre portfolio 
    else:
        facilities_add_p = facilities_add_p.withColumn('imp_maximo_individual', F.least(*[F.col('imp_maximo_'+x) for x in l_pr])
                                                      ).withColumn('limit_individual_p',F.least(*[F.col('limit_'+x) for x in l_pr])) # el importe y limite más restrictivo de los limites % sobre portfolio 

limite con importe sujeto a portfolio size: rating_rga_esl
solo un limite individual con limite de importe portfolio


In [79]:
# LIMITES INDIVIDUALES comunes (el % va sobre el portfolio)
facilities_add_p.select(*[F.col(x).cast('float') for x in limits_indv]).show(5,False)

Py4JJavaError: An error occurred while calling o964.showString.
: org.apache.spark.SparkException: Could not execute broadcast in 300 secs. You can increase the timeout for broadcasts via spark.sql.broadcastTimeout or disable broadcast join by setting spark.sql.autoBroadcastJoinThreshold to -1
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:205)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:515)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeBroadcast$1(SparkPlan.scala:193)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.joins.BroadcastHashJoin

In [ ]:
# LIMITES INDIVIDUALES portfolio
facilities_add_p.groupBy(*[F.col('limit_'+c)for c in l_pr],'imp_maximo_individual').count().show(100,False)

In [ ]:
facilities_tot = facilities_add_p.withColumn('limit_individual', F.least(*[F.col(x).cast('float') for x in limits_indv])
                              ).withColumn('importe_titulizable_ini', F.col('importe_susceptible') * F.col('limit_individual')
                              ).withColumn('importe_titulizable', F.when(F.col('importe_titulizable_ini')<=F.col('imp_maximo_individual'),F.col('importe_titulizable_ini')
                                                                        ).otherwise(F.col('imp_maximo_individual'))
                              ).withColumn('excluded', F.when(((F.col('limit_individual')==0)|(F.col('imp_maximo_individual')<=0)),1).otherwise(F.col('excluded'))
                              ).withColumn('candidata',F.when(F.col('importe_titulizable')>0,1).otherwise(0))                             

In [ ]:
facilities_tot.select(*key_facility,*limits_indv,'limit_individual','importe_susceptible','importe_titulizable_ini','importe_titulizable','imp_maximo_individual','excluded','exclusion_limit','candidata').show(10,False)

### Añado columna Motivo Exclusion
para poder analizar el por qúe se excluye la facility
- hay casos de importe susceptible en negativo, lo tomamos como importe 0

In [ ]:
facilities_tr = facilities_tot.withColumn('motivo_exclusion', F.when(F.col('excluded')==1,'limite individual 0'
                                                         ).when(F.col('importe_susceptible')<=0,'importe susceptible 0'
                                                         ).otherwise('NA')
                              ).withColumn('detalle_exclusion', F.col('exclusion_limit'))

In [ ]:
facilities_tr.groupBy('candidata','motivo_exclusion').count().show()

In [ ]:
# comprobar caso anómalo
# facilities_tr.where((F.col('candidata')==0)&(F.col('motivo_exclusion')=='NA')
#                    ).select('limit_rating_rga_esl','importe_titulizable_ini','importe_titulizable','imp_maximo_individual').show(5,False)

In [ ]:
# facilities_tr.groupBy('candidata','motivo_exclusion').count().show()
# +---------+--------------------+-----+
# |candidata|    motivo_exclusion|count|
# +---------+--------------------+-----+
# |        0| limite individual 0|  254|
# |        1|                  NA| 1207|
# |        0|importe susceptib...|  155|
# +---------+--------------------+-----+

In [ ]:
# facilities no excluidas por los limites individuales
facilities_tr.where(F.col('excluded')==0).select(*key_facility,*limits_indv,'limit_individual',
                                                 'importe_susceptible','importe_titulizable','imp_maximo_individual',
                                                 'excluded','exclusion_limit').show(10,False)

### Persistimos Sbx
persistimos en sandbox las facilities excluidas de la cartera a titulizar

In [ ]:
path_facilities_lim_ind = root_pathc + '/facilities_limit_ind'
path_facilities_lim_ind

In [ ]:
facilities_tr.write.parquet(path_facilities_lim_ind, mode='overwrite')

#### Estadisticas

In [ ]:
df = dataproc.read().parquet(path_facilities_lim_ind)

In [ ]:
print('facilities totales', df.count())
print('facilities excluidas por limites individuales', df.where(F.col('excluded')==1).count())
print('facilities con importe titulizable', df.where(F.col('candidata')==1).count()) # en la ultima titulización finalmente fueron 86 las facilities seleccionadas

In [ ]:
# pueden darse importes susceptibles negativos, pero se marcan como NO CANDIDATAS a entrar en la cartera
# df.where(F.col('importe_titulizable')<0).show()
# df.where(F.col('importe_titulizable')<0).select('candidata').distinct().show()

#### Facilities excluded
persistimos las facilities excluidas por limites individuales
(*) ahora mismo en el visor, ver si seguimos dejando esta parte

In [ ]:
path_excluded = root_pathc + '/facilities_excluded'
path_excluded

In [ ]:
# cols_datio

In [ ]:
cols_ex = set(key_facility + cols_datio)
# cols_ex

In [ ]:
# cols_ex = cols_traza + key_facility + cols_datio
df_exc = df.where(F.col('excluded')==1).select(*cols_traza,*cols_ex)
df_exc.show(2,False)

In [ ]:
# df_exc.groupBy('customer_country').count().show()

In [ ]:
# sorted(df_exc.columns)

In [ ]:
df_exc.write.parquet(path_excluded, mode='overwrite')

In [ ]:
# el expiration_date es menor a un año respecto a la fecha de la titulizacion '2024-06-24'
# df_exc.select('expiration_date','exclusion_limit').where(F.col('exclusion_limit').like('%maturity_min%')).show(100,False)

# 4. Limites Portfolio
Una vez sabemos lo disponible de cada préstamo, seleccionamos la cartera de los que pueden entrar
- Limites Portfolio:ordenamos facilities y vamos consumiendo los límites globales
- Resultado final: colectivo óptimo de facilities a titulizar

Marcan el % sobre el portfolio_size que se puede consumir de cada categoria

In [ ]:
# Quitamos lo limites marcados como cabecera (no interfieren en el ranking) y el sts
# limit_cabecera = ['portfolio_size','portfolio_type','portfolio_date','sts','ccf','rw']
# antes excluiamos los 0% porque estaban ya analizados .where(F.col('limit_value')!=0)

limit_port = limites_total.where(F.col('limit_scope')=='portfolio'
                            ).where(F.col('header_flag')==0
                            ).where(F.col('limit_type')!='sts')

                        # ).where(~(F.col('limit_type').isin(*limit_cabecera)))
limit_port.show(2,False)

In [ ]:
print('limites portfolio:', limit_port.count())

## Diccionarios
Generamos diccionarios con los datos de los limites

### Diccionario limite-campo Datio
correspondencia entre: limite - campo Datio
- 'customer_subsector': 'g_asset_allocation_subsec_desc'

In [ ]:
# limites globales
lista =limit_port.select('limit_type','campo_datio').collect()
dict_lim_port = {row['limit_type']: row['campo_datio'] for row in lista}

In [ ]:
# dict_lim_port

In [ ]:
lim_portfolio = list(dict_lim_port.keys())
# lim_portfolio

### Diccionario limite-valores
Limites del launchpad: correspondencia entre: limite - categoría - valor
- 'customer_country-Spain': 0.3499999940395355

In [ ]:
# limites globales: no complejos
lista1 =limit_port.select('limit_type','concept1_value','limit_value'
                         ).where(F.col('complex_limit')==0).collect()
dict_lim_values1 = {row['limit_type'] + '-' + row['concept1_value']:row['limit_value'] for row in lista1}

In [ ]:
# sorted(dict_lim_values1)

In [ ]:
# limites globales: complejos
lista2 =limit_port.select('limit_type','concept2_value','limit_value'
                         ).where(F.col('complex_limit')==1).collect()
dict_lim_values2 = {row['limit_type'] + '-' + row['concept2_value']:row['limit_value'] for row in lista2}

In [ ]:
lista2

In [ ]:
# sorted(dict_lim_values2)

In [ ]:
# FUSIONO en un único diccionario
dict_lim_values= dict_lim_values1.copy()   # start with x's keys and values
dict_lim_values.update(dict_lim_values2) 

In [ ]:
# sorted(dict_lim_values)

## Facilities disponibles
- listado de facilities con traza de importe titulizable + limites individuales

(*) cogemos todo el listado para tener traza de todo por si hay que revisar algún límite. Sólo entrarían en la cartera a titulizar las candidatas = 1 (importe_titulizable>0)

In [ ]:
path_facilities_lim_ind = root_pathc + '/facilities_limit_ind'
path_facilities_lim_ind

In [ ]:
# Pto de arranque son las facilities con importe titulizable
facilities_disp = spark.read.parquet(path_facilities_lim_ind)
print('facilities totales:',facilities_disp.count())
print('facilities candidatas a titulizar:',facilities_disp.where(F.col('candidata')==1).count())

In [ ]:
facilities_disp.show(2,False)

In [ ]:
# facilities_disp.columns

### Preparación datos
Transformados en Pandas

In [ ]:
raw_data = facilities_disp.toPandas() 

#### Formateo de los datos
Formatos correspondientes pyspark - pandas

In [ ]:
# tipos al pasar a pandas
# raw_data.dtypes

In [ ]:
# Diccionario de tipologia PYSPARK de las columnas

# listado de: columna - tipo de dato
tipos = list(facilities_disp.dtypes)

# en formato diccionario
cols_type = dict(tipos)
# cols_type

In [ ]:
# LOOP PARA FORMATEAR PANDAS COLUMNAS DATIO DEL DICCIONARIO
# posibles valores: 'string','date','decimal(,)','double','int','boolean'
for r in raw_data.columns:
    if (cols_type[r] == 'string'):
        # print('string',r)
        raw_data[r] = raw_data[r].astype('str')
    elif (cols_type[r] == 'date'):
        # raw_data[r] = raw_data[r].astype('datetime64')
        raw_data[r] = raw_data[r].astype('datetime64[D]')
        #print(r)
    elif (cols_type[r] == 'boolean'):
        raw_data[r] = raw_data[r].astype('bool')
    elif (cols_type[r] == 'int'):
        raw_data[r] = raw_data[r].astype('int')
    elif (cols_type[r] == 'double'):
        raw_data[r] = raw_data[r].astype('float')
    elif ('decimal' in cols_type[r]):
        raw_data[r] = raw_data[r].astype('float')       

In [ ]:
df = raw_data.sort_values(by='importe_titulizable', ascending=False).reset_index(drop=True)

#### Columnas limites portfolio
- Añadimos una columna para marcar la traza de los valores de limite portfolio que aplican por facility (por defecto 1.0)
- selected = 1: si se ha seleccionado para la titulizacion (por defecto 0)
- limit_portfolio = min(limites portfolio) que aplica a la facility (por defecto 1.0, un 100% de lo disponible) 

In [ ]:
def ini_columns():
    cols_test=[]
    
    for k in lim_portfolio:
        df['limit_'+k]=1.0000 # por defecto el limite es del 100%, en el proceso de cartera en los casos que hay limite marcado se actualiza
        df['max_portfolio_size_'+k]= portfolio_size #por defecto, se puede coger el máximo importe a titulizar
        df['consumido_'+k]=0.0000 # % consumido para ese limite
        df['importe_consumido_'+k]=0.0000 # importe consumido para ese limite
        cols_test.append('limit_'+k)
        cols_test.append('max_portfolio_size_'+k)
        cols_test.append('consumido_'+k)
        cols_test.append('importe_consumido_'+k)

    df['selected']=0 # si se incluye en la cartera
    df['limit_portfolio']=1.0000 # limite más restrictivo a nivel portfolio
    df['porcentaje_portfolio_size']= 0.0000 # % del importe del portfolio_size consumido por la facility (peso de la facility en la cartera)
    df['importe_optimo']= 0.0000 # importe que se incluye de la facility
    df['ranking_candidata']=0 # orden de prioridad en el modelo
    df['ranking_selected']=0 # orden de selección en la cartera final
    df['importe_optimo_acumulado']=0.0000 # suma de los importes de las facilities incluidas en la cartera
    df['porcentaje_portfolio_size_acumulado']=0.0000 # % de la cartera (portfolio_size) que se ha ido consumiendo
    df['porcentaje_optimo']=0.0000 # % sobre el importe suceptible de la facility que se toma en la cartera
    
    # lo incluyo como columna el máximo importe a titulizar
    df['limit_portfolio_size']=portfolio_size

    return cols_test + ['selected','limit_portfolio','porcentaje_portfolio_size','importe_optimo',
                        'ranking_candidata','ranking_selected','importe_optimo_acumulado',
                        'porcentaje_portfolio_size_acumulado','porcentaje_optimo','limit_portfolio_size']
    
cols_test = ini_columns()

In [ ]:
df[[*cols_test]]

### Consumos Limites
- Por cada limites-categoria posible de las facilities:
    - columna del consumo acumulado: Inicializamos a 0 consumo inicial
    - columna del máximo permitido: Calculamos el máximo sobre el portfolio size (max_limite = portfolio_size * limite_launchpad)

- Ej. posibles monedas: EUR,DOL,.. se irán incrementando al seleccionar préstamos con esa moneda y contrastando con su máximo

(*)Todos los posibles valores a nivel facility se tienen que corresponder con un limite,
si no existe se genera por defecto con 1.0, que es el máximo

In [ ]:
# dict_lim_port.items()

In [ ]:
#dict_lim_values['maturity_min-365'] # 0.0

In [ ]:
# sorted(list(dict_lim_values.keys()))

#### Diccionarios
- con los consumos a 0
- con el consumo_maximo calculado (portfolio_size*limite)

In [ ]:
def inicializar_consumos():
    l_lim_consumidos= {}# inicializamos diccionario de consumos
    l_lim_marcados={} # inicializamos diccionario de limites
    l_max_limites={} # calculo el máximo portfolio por cada limite-categoria
    limit_keys = list(dict_lim_values.keys()) # % marcados en launchad
    keys_fechas = ['maturity_min','maturity_max']

    for k,v in dict_lim_port.items():
        # print(k,v)
        if(k not in keys_fechas): # caso base: limite-categoria=valor      
            for k1 in df[v].unique(): # recojo todos los posibles valores de ese campo en la facility
                l_lim_consumidos[k+'-'+str(k1)]=0.0000

                #si no hay marcado limite se pone a 1.0
                if(k+'-'+str(k1) not in limit_keys):                 
                    valor = 1.0000

                    if(k+'-'+'total' in limit_keys): # si es un limite a nivel global como TODOS los grupos
                        valor = round(float(dict_lim_values[k+'-'+'total']),4)

                    dict_lim_values[k+'-'+str(k1)]=valor 
                    
        else: # no es un limite directo y hay que calcular fechas: limite-ndias=valor
            # print('limite:',k)
            lk = [l for l in limit_keys if k in l][0]
            dias = int(lk[len(k)+1:]) # ndias que marcan fecha tope
            f_tope = np.datetime64(get_fecha(date_titulizacion,dias)).astype('datetime64[D]')
            
            for k1 in df[v].unique().astype('datetime64[D]'):
                l_lim_consumidos[k+'-'+str(k1)]=0.0000                
                # maturity_min: si la fecha maturity es mayor o igual no aplica limite
                # maturity_max: si la fecha maturity es menor o igual no aplica limite
                if(((k=='maturity_min')&(k1>=f_tope)) | ((k=='maturity_max')&(k1<=f_tope))):
                    valor = 1.0000 # valor si el limite no aplica
                else:
                    valor = round(float(dict_lim_values[lk]),4) # valor si el limite aplica
                    # print('fecha restringida:',k1, 'valor:',valor)
                    
                dict_lim_values[k+'-'+str(k1)]=valor


    facilities_keys = list(l_lim_consumidos.keys()) 
    l_lim_marcados = {key: round(dict_lim_values[key],4) for key in facilities_keys}
    l_max_limites = {key: (l_lim_marcados[key] * portfolio_size)  for key in l_lim_marcados}
    return l_lim_consumidos,l_lim_marcados,l_max_limites


    
l_lim_consumidos,l_lim_marcados,l_max_limites= inicializar_consumos()
# generamos un diccionario tb de importe consumido por cada limite
l_importe_consumidos = l_lim_consumidos.copy()

In [ ]:
print('listado de clave limites_categoria marcados en el lauchpad:',sorted(l_lim_marcados))

In [ ]:
key_l = list(l_lim_marcados)[0]
key_l

In [ ]:
l_lim_marcados[key_l]

In [ ]:
l_lim_consumidos[key_l]

In [ ]:
l_max_limites[key_l]

In [ ]:
l_lim_marcados['customer_country-Austria']

In [ ]:
l_max_limites['customer_country-Austria']

In [ ]:
l_lim_marcados['sts_group-G00000000000005']

In [ ]:
l_max_limites['sts_group-G00000000000005']

In [ ]:
l_lim_marcados['group-G20080109113719']

In [ ]:
l_max_limites['group-G20080109113719']

In [ ]:
# dict_lim_values['customer_country-Brazil']
# l_lim_marcados['customer_country-Brazil']
# l_lim_marcados['maturity_min-2025-02-05']
# l_lim_marcados['maturity_min-2026-10-13']

In [ ]:
#sorted(l_lim_marcados)

In [ ]:
# l_lim_consumidos

# 'financial_product': {'Term Loan': 0,
#   'RCF': 0,
#   'Credit Line and Guarantee': 0,
#   'Guarantee': 0}

In [ ]:
# PRUEBA para limites en concreto
# divisa_importe = {divisa: 0 for divisa in df['currency_id'].unique()} # posibles valores de divisas en nuestras facilities
# pais_importe = {pais: 0 for pais in df['customer_country'].unique()} # posibles valores de paises en nuestras facilities

# divisa_importe

## Generación Cartera
Nos quedamos con las facilities que entran en la titulizacion

### Limites por facility
Función para obtener campo-valor de la facility a contrastar con los limites marcados
- 'customer_country-United States of America'

In [ ]:
# guardamos en un diccionario columna Datio - valor para la facility analizada
def get_keys_facility(i_facility, dataframe):   
    d_keys={} # formato lista de clave-valor
    res={} # diccionario de campos
    
    # para cada limite, el valor del campo de Datio indicado 
    # ej. para facility fila 1 -> divisa = df.loc[1, 'currency_id'] -> 'divisa':'USD'
    for k,v in dict_lim_port.items():    
        res[k]=dataframe.loc[i_facility, v]
        
        if (cols_type[v] == 'date'):
            res[k]=dataframe.loc[i_facility, v].strftime('%Y-%m-%d')
        
    for k,v in res.items():
        key = str(k) + '-' + str(v)
        d_keys[k]=key
        
    return d_keys

In [ ]:
dict_lim_port.items()

In [ ]:
#get_keys_facility(0,df)

In [ ]:
# get_keys_facility(0, df)

# {'customer_subsector': 'customer_subsector-Integrated Utility',
#  'customer_country': 'customer_country-United States of America',
#  'customer_sector': 'customer_sector-Utilities',
#  'financial_product': 'financial_product-Term Loan',
#  'non_ig': 'non_ig-0',
#  'sts_group': 'sts_group-G00000000010863',
#  'divisa': 'divisa-USD',
#  'group': 'group-G00000000010863'}

In [ ]:
# get_keys_facility(54, df)

**dudas de como aplicar el limite rw (usa para ponderar la ordenación por riesgo y es 0.9 sobre el RORC)**

In [ ]:
# columnas candidatas sobre las que aplican los %
# cols_ajuste = ['Total_Amount_EUR','Total_Amount_CCY','EC_per','RC_per','Risk_Weight','EL_per','Reg_EL_per']

### Ordenación Facilities
- Ordenamos de mayor a menor importe a titulizar

(*) segunda versión incluir tb ordenar de mayor a menor riesgo

In [ ]:
# columnas de ordenacion: importe titulizable del préstamo. segunda fase probar tb a incluir el porcentaje de riesgo
cols_ord = ['importe_titulizable']#, 'RC_per']

df = df.sort_values(by=cols_ord, ascending=False).reset_index(drop=True)

# ** parte inicialmente no requerida ***#
# HACERLO PARA TODOS LOS CAMPOS NECESARIOS EN LOS LIMITES (con el diccionario de limites)
# Convertir datos a listas para su uso en Pulp y OR-Tools
# importes = data['Total_Amount_EUR'].tolist()
# riesgos = data['RC_per'].tolist()
# divisas = data['currency_id'].tolist()
# paises = data['customer_country'].tolist()
# ranking = data.index.values.tolist()
# n = len(importes)


In [ ]:
# df[[*key_facility,*cols_ord,'candidata']]

### Inicialización de variables
Inicializamos las variables necesarias en el modelo de selección

In [ ]:
# Inicializar el contador del importe acumulado de las facilities
importe_acumulado = 0.0000
# Inicializamos el % carterizado sobre el total a titulizar
porcentaje_portfolio_size_acumulado = 0.0000000

# marcamos el máximo a titulizar, que es el importe que se quiere titulizar
maximo_importe = portfolio_size

# columna sobre el importe de la facility disponible a ser titulizado
col_imp = 'importe_titulizable'

# Lista para almacenar los índices de las facilities seleccionadas y sus proporciones
selected_facilities = []

# los contadores de limites
# l_lim_consumidos: todos a 0 inicialmente, l_lim_marcados: % marcados en el launchad, l_max_limites: portfolio_size por limite
l_lim_consumidos,l_lim_marcados,l_max_limites = inicializar_consumos() # inicializamos los consumos antes de comenzar

# columnas añadidas en el proceso
cols_test = ini_columns() # inicializamos los valores de las columnas

### Modelo de titulizacion
Proceso para generar la cartera

In [ ]:
# si queremos ver el detalle de la traza de ciertas facilities
traza_candidata = []#['830990','815808','848947']#[]#['185065','783009']
traza_exclusion = True # para almacenar traza en la cartera final

In [ ]:
for i in df.index: # para recoger cada numero de fila (1 fila por facility)
    if(df.loc[i]['delta_file_id'] in traza_candidata):
        print('**** facility ',i,'****************************************************')
        print('---- id_facilitiy',df.loc[i]['delta_file_id'])
    
    # PASO_1: importe titulizable de la facility
    expediente_importe = df.loc[i, col_imp]
    df.loc[i,'ranking_candidata']=i+1 # posicion en el ranking de la facility dentro de las candidatas en la cartera (iniciamos en 1)
    
    if(df.loc[i]['delta_file_id'] in traza_candidata):
        print('expediente_importe',expediente_importe)
    
    
    # PASO_2: %máximo a titulizar de la facility    
    # inicializamos el máx titulizable de la facility
    max_proportion = 1.0000 # por defecto se intenta titulizar todo lo disponible por facility
    
    # si el importe de la facility ya excede lo que hay disponible para titulizar se calcula ese máximo
    if((importe_acumulado + expediente_importe) > maximo_importe):
        max_proportion = (maximo_importe - importe_acumulado) / expediente_importe # max % que se puede titulizar de la facility
    
    if(df.loc[i]['delta_file_id'] in traza_candidata):
        print('maximo:',max_proportion)
        
    
    # PASO_3: %mínimo marcado por los limites que será lo máximo que se puede titulizar
    # campos de la facility - valor que hay que limitar
    keys_f = get_keys_facility(i,df)
    if(df.loc[i]['delta_file_id'] in traza_candidata):
        print(keys_f)
    min_lim= max_proportion # inicializamos el mínimo con el máximo posible a titulizar
    min_portfolio=max_proportion
    for k,v in keys_f.items():
        #print('limite aplicar:',l_lim_marcados[v])
        df.loc[i, 'limit_'+k] = l_lim_marcados[v] # genero el limite del campo y actualizo valor para la categoría de esa facility
        df.loc[i,'max_portfolio_size_'+k]= l_max_limites[v]
        min_portfolio = min(min_portfolio,l_lim_marcados[v])# minimo de los limites de portfolio       
        disponible_proportion = round(float(l_lim_marcados[v] - l_lim_consumidos[v]),4)
        #***********************
        if(disponible_proportion<0): # debido al redondeo de decimales
            disponible_proportion = 0.0000
            # print('limit_'+k,l_lim_marcados[v])
            # print('consumido',l_lim_consumidos[v])
            # print(i)
        #***********************
        df.loc[i, 'disponible_'+k] = disponible_proportion
        min_lim = min(min_lim,disponible_proportion) # minimo que se puede aplicar teniendo en cuenta lo que ya está consumido   
        if(df.loc[i]['delta_file_id'] in traza_candidata):
            print('limite-categoria',v,'% máximo',min_lim, 'marcado',l_lim_marcados[v],'consumido',l_lim_consumidos[v], 'disponible',disponible_proportion)
            
    if(df.loc[i]['delta_file_id'] in traza_candidata):
        print(' proporcion máxima:',min_lim)
    df.loc[i, 'limit_portfolio'] = min_portfolio
    df.loc[i, 'porcentaje_max_portfolio'] = min_lim # máximo % disponible de los consumos de los limites
    
    # PASO_4: Comprobamos si esa facility se puede incluir y sino pasamos a la siguiente
    # que sea candidato por importe titulizable>0 y que no se haya consumido alguno de sus limites
    if((df.loc[i, 'candidata']==1) and (min_lim>0)):
       
        # PASO_5: Calculamos el importe a titulizar de la facility
        importe_max_facility = maximo_importe * min_lim # importe máximo a titulizar para la facility: portfolio_size*%más restrictivo
        df.loc[i,'importe_max_facility'] = importe_max_facility # guardamos el importe máximo que se puede coger de esa facility
        # si el importe que se puede incluir es mayor o igual al de la facility incluimos todo
        if(importe_max_facility >= expediente_importe):
            importe_seleccionado = expediente_importe
        else:
            importe_seleccionado = importe_max_facility
        if(df.loc[i]['delta_file_id'] in traza_candidata):
            print('importe_seleccionado:',importe_seleccionado)

         # Si el importe óptimo>0, 
        if(importe_seleccionado>0):
            # PASO_6: Actualizamos los acumulado tanto de limites como de importe a titulizar
            importe_acumulado = importe_acumulado + importe_seleccionado
            por_consumido = round(float(importe_seleccionado/maximo_importe),7) # peso_cartera: % de la titulizacion sobre la cartera
            if(df.loc[i]['delta_file_id'] in traza_candidata):
                print('% usado de la cartera (porcentaje_portfolio_size):',por_consumido)
                
            for k,v in keys_f.items():
                l_lim_consumidos[v] = round(float(l_lim_consumidos[v] + por_consumido),7) # actualizo los consumos limite-categoria
                l_importe_consumidos[v] = l_importe_consumidos[v] + importe_seleccionado
                df.loc[i, 'consumido_'+k] = l_lim_consumidos[v]
                df.loc[i, 'importe_consumido_'+k] = l_importe_consumidos[v]
                if(df.loc[i]['delta_file_id'] in traza_candidata):
                    print('consumido ',v,':',l_lim_consumidos[v])
                    print('importe_consumido ',v,':',l_importe_consumidos[v])

            # PASO_7: Rellenamos traza a nivel facility
            # Marcamos la facility como seleccionada con su proporcion_optima y su importe_optimo
            selected_facilities.append((i, por_consumido))  
            df.loc[i,'selected']=1
            df.loc[i,'importe_optimo'] = importe_seleccionado
            df.loc[i,'porcentaje_optimo']= round(float(importe_seleccionado/df.loc[i,'importe_susceptible']),7) # % del importe susceptible que finalmente se coge
            df.loc[i,'ranking_selected']=len(selected_facilities) # posicion en el ranking de las facilities seleccionadas
            df.loc[i,'importe_optimo_acumulado']= importe_acumulado
            porcentaje_portfolio_size_acumulado = round(float(importe_acumulado/maximo_importe),7) # % usado del portfolio_size
            df.loc[i,'porcentaje_portfolio_size_acumulado']= porcentaje_portfolio_size_acumulado
            df.loc[i, 'porcentaje_portfolio_size'] = por_consumido # porcentaje del portfolio_size que se consume (% sobre la cartera final)
            
            # TRAZA DE SALIDA
            if(df.loc[i]['delta_file_id'] in traza_candidata):
                print('---- facility seleccionada:',df.loc[i,'ranking_selected'])
                min_disponible=1.0
                print('***Se ha alcanzado un limite a nivel columna-categoria. NUM CANDIDATA: ***',i+1)
                for k,v in keys_f.items():
                    print('----- limite analizado:',v)
                    print('limite en launchpad:',l_lim_marcados[v])
                    print('consumido:',l_lim_consumidos[v])      
                    disponible_proportion = round(float(l_lim_marcados[v] - l_lim_consumidos[v]),4)
                    if(disponible_proportion<0): # debido al redondeo de decimales
                        disponible_proportion = 0.0000
                    print('disponible:',disponible_proportion)
                    min_disponible = min(min_disponible,disponible_proportion)  
                print('máximo disponible para la facility:',min_disponible)
                
    # para aquellas facilities que no entran dejamos traza y arrastran los consumos e importes acumulados hasta el momento
    else:
        if(df.loc[i, 'candidata']==1): #solo para aquellas candidatas (que tengan importe disponible)
            min_disponible=1.0
            print('***Se ha alcanzado un limite a nivel columna-categoria. NUM CANDIDATA: ',i+1, 'DELTA FILE:',df.loc[i]['delta_file_id'],'***')
            for k,v in keys_f.items():    
                disponible_proportion = round(float(l_lim_marcados[v] - l_lim_consumidos[v]),4)
                if(disponible_proportion<0): # debido al redondeo de decimales
                    disponible_proportion = 0.0000
                if(disponible_proportion==0):
                    print('----- limite analizado:',v)
                    print('limite en launchpad:',l_lim_marcados[v])
                    print('consumido:',l_lim_consumidos[v]) 
                    print('disponible:',disponible_proportion)
                    min_disponible = min(min_disponible,disponible_proportion)
                    if(l_lim_marcados[v]==0):
                        df.loc[i, 'motivo_exclusion']='limite portfolio 0'
                        df.loc[i,'detalle_exclusion']=v
                    else:
                        df.loc[i, 'motivo_exclusion']='consumido limite portfolio'
                        df.loc[i,'detalle_exclusion']=v                            
            print('máximo disponible para la facility:',min_disponible)
            
        # arrastramos valores acumulados
        df.loc[i,'importe_optimo_acumulado']= importe_acumulado # importe acumulado hasta el momento
        df.loc[i,'porcentaje_portfolio_size_acumulado']= porcentaje_portfolio_size_acumulado # % usado del portfolio_size hasta el momento
        for k,v in keys_f.items():
            df.loc[i, 'consumido_'+k] = l_lim_consumidos[v]
            df.loc[i, 'importe_consumido_'+k] = l_importe_consumidos[v]
        
    
    # PASO_8: si se ha alcanzado el máximo a titulizar salimos del bucle, ya tenemos las facilities a titiulizar
    if(importe_acumulado>=maximo_importe):
        print('***Se ha superado el portfolio size. Importe acumulado:',importe_acumulado)
        break;

In [ ]:
df[[*cols_test,'importe_titulizable','porcentaje_max_portfolio','porcentaje_optimo']]

In [ ]:
# (df['selected']==1).sum()
# df.count()

In [ ]:
#l_lim_marcados['customer_country-United States of America']

In [ ]:
#l_lim_consumidos['customer_country-United States of America']

# 5. Persistimos
- Pasamos de pandas a pyspark para guardar el resultado

In [ ]:
df_sp = dataproc.getSparkSession().createDataFrame(df).fillna(0)

## Ajustes de tipos Columnas

In [ ]:
tipos = []
for c in df_sp.columns:
    tipo = df_sp.schema[c].dataType
    # print(c,tipo)
    if tipo not in tipos:
        tipos.append(tipo)
        
    # redondeo los tipos numéricos a 2 decimales    
    if str(tipo) in ('LongType','DoubleType'):
        # print(c)
        if(c in ['porcentaje_portfolio_size','porcentaje_portfolio_size_acumulado']):
            df_sp = df_sp.withColumn(c, F.round(F.col(c),7)) #dejamos un redondeo de 6 decimales 
        else:
            df_sp = df_sp.withColumn(c, F.round(F.col(c),4)) #dejamos un redondeo de 4 decimales 
                                       
print('tipo de datos de las columnas:',tipos)

### parquet

In [ ]:
path_out = root_path+'closing_date='+str(fecha)+'/cartera_titulizar'
path_out

In [ ]:
df_sp.write.parquet(path_out,mode='overwrite')

### csv

In [ ]:
path_out_csv = root_path+'closing_date='+str(fecha)+'/cartera_titulizar_csv'
path_out_csv

In [ ]:
dataproc.read().parquet(path_out).coalesce(1).write.csv(path_out_csv, mode='overwrite', header='True')

# 6. Estadísticas

In [ ]:
fecha_ejecucion = last_partition (root_path, 'closing_date')
print('Fecha de ejecución del modelo titulización:', fecha_ejecucion)
root_pathc = root_path + 'closing_date=' + str(fecha_ejecucion)
root_pathc

In [ ]:
path_out = root_pathc+'/cartera_titulizar'
path_out

In [ ]:
df_total = dataproc.read().parquet(path_out)
cartera_candidatas = df_total.where(F.col('candidata')==1)
cartera_titulizar = df_total.where(F.col('selected')==1)

In [ ]:
df_total.show(2,False)

In [ ]:
key_t = ['delta_file_id','delta_file_band_id']
n= df_total.groupBy(*key_t).agg(F.count('delta_file_id').alias('n')).where(F.col('n')>1).count()
if (n>1):
    print('MicroStrategy: Hay duplicados a nivel expediente-tramo')
    print(n)

In [ ]:
ids_delta = ['185065','783009']

In [ ]:
df_total.where(F.col('delta_file_id').isin(ids_delta)).show(2,False)

In [ ]:
key_t = ['delta_file_id','delta_file_band_id','branch_id']
if (df_total.groupBy(*key_t).agg(F.count('delta_file_id').alias('n')).where(F.col('n')>1).count()>1):
    print('Hay duplicados a nivel facility')

In [ ]:
# df_total.printSchema()

In [ ]:
# sorted(df_total.columns)

In [ ]:
print('numero total de facilities', df_total.count())
print('numero de facilities candidatas a titulizar', cartera_candidatas.count())
print('numero de facilities en la cartera a titulizar', cartera_titulizar.count())

In [ ]:
df_importes = df_total.groupBy().agg(F.sum('importe_susceptible').alias('importe_susceptible'),
                 F.sum('importe_titulizable').alias('importe_titulizable'),
                 F.sum('importe_optimo').alias('importe_final_titulizar'),
                 F.collect_set('limit_portfolio_size')[0].alias('portfolio_size'))                

In [ ]:
suma_sus,suma_imp_d,suma_imp,importe_titulizar = [(x.importe_susceptible,x.importe_titulizable,x.importe_final_titulizar,x.portfolio_size) for x in df_importes.collect()][0]

In [ ]:
# suma_imp_d = [x.importe_titulizable for x in cartera_candidatas.groupBy().agg(F.sum('importe_titulizable').alias('importe_titulizable')).collect()][0]

In [ ]:
# suma_imp = [x.importe_final_titulizar for x in cartera_titulizar.groupBy().agg(F.sum('importe_optimo').alias('importe_final_titulizar')).collect()][0]

In [ ]:
# importe_titulizar = [x.limit_portfolio_size for x in df_total.select('limit_portfolio_size').distinct().collect()][0]

In [ ]:
print('importe que se quiere titulizar',importe_titulizar) # 2.000.000.000
print('importe susceptible a titulizar',suma_sus) # 63.917.592.977
print('importe total titulizable', suma_imp_d) # 3.539.030.085
print('importe final titulizado', suma_imp)  # 793.488.411
print('porcentaje titulizado del total a titulizar', str(suma_imp/importe_titulizar)) # 0.39
print('se deja sin titulizar', str(importe_titulizar - suma_imp)) # 1.206.511.588

### Trazas

#### importe susceptible
- importe1 = saldo vivo+importe disponible*CCF
- importe2 = importe_titulizado/risk_retention
- importe_susceptible = min(ead_regulatorio,importe1) - importe2

In [ ]:
c_cal = ['bbva_drawn_eur_amount','bbva_available_eur_amount','limit_ccf','importe1',
          'gf_facility_securitization_amount','limit_risk_retention','importe2',
          'gf_ma_ead_amount','importe_susceptible']

In [ ]:
df_total.select(*key_limites,*key_facility,*c_cal).show(10,False)

#### limites individuales

In [ ]:
c_lim_ind = ['limit_rating_sp','limit_excluded_facilities','limit_risk_retention',
             'limit_sts_rw_modelo','limit_sts_payment','limit_individual']
c_imp = ['importe_susceptible','importe_titulizable']
c_ex = ['excluded','exclusion_limit']
c_flag_i = ['candidata']

In [ ]:
df_total.select(*key_limites,*key_facility,*c_lim_ind,*c_imp,*c_ex,*c_flag_i).show(10,False)

#### limites portfolio

In [ ]:
c_flag_p = ['selected','ranking_candidata','ranking_selected']
c_lim_portfolio =['limit_customer_subsector','consumido_customer_subsector',
                  'limit_customer_country','consumido_customer_country',
                  'limit_customer_sector','consumido_customer_sector',
                  'limit_financial_product','consumido_financial_product',
                  'limit_non_ig','consumido_non_ig',
                  'limit_sts_group','consumido_sts_group',
                  'limit_divisa','consumido_divisa',
                  'limit_group','consumido_group',
                  'limit_portfolio']
c_imp_p =['importe_titulizable','porcentaje_portfolio_size','importe_optimo',
         'porcentaje_portfolio_size_acumulado','importe_optimo_acumulado']


In [ ]:
df_total.where(F.col('candidata')==1 # para analizar las facilities que son candidatas en la titulizacion 
              ).select(*key_limites,*key_facility,*c_lim_portfolio,*c_imp_p,*c_flag_p).show(10,False)